In [1]:
%gui qt

import pyqtgraph as pg
import pyqtgraph.metaarray as metaarray
from pyqtgraph.Qt import QtGui, QtCore
import pyqtgraph.opengl as pgl
import scipy.ndimage as ndi
import numpy as np

pg.mkQApp()

In [2]:
drive_path = '/Volumes/Brain2016'
import os
from allensdk.core.mouse_connectivity_cache import MouseConnectivityCache
import pandas as pd


In [3]:
# When downloading 3D connectivity data volumes, what resolution do you want (in microns)?  
# Options are: 10, 25, 50, 100
resolution_um=25

# The manifest file is a simple JSON file that keeps track of all of
# the data that has already been downloaded onto the hard drives.
# If you supply a relative path, it is assumed to be relative to your
# current working directory.
manifest_file = os.path.join(drive_path, "MouseConnectivity","manifest.json")

mcc = MouseConnectivityCache(manifest_file=manifest_file, resolution=resolution_um)

In [4]:
template, template_info = mcc.get_template_volume()

In [ ]:
view = pgl.GLViewWidget()

# atlas = metaarray.MetaArray(file='ccf.ma', readAllData=True)

# img = np.ascontiguousarray(atlas.asarray()[::8,::8,::8])

In [25]:
img = template[::4,::4,::4]
pad = 50
padded_img = np.zeros((img.shape[0]+pad, img.shape[1]+pad, img.shape[2]+pad), dtype=img.dtype)
padded_img[pad/2:pad/2+img.shape[0], pad/2:pad/2+img.shape[1], pad/2:pad/2+img.shape[2]] = img

In [26]:
# render volume
#vol = np.empty(img.shape + (4,), dtype='ubyte')
#vol[:] = img[..., None]
#vol = np.ascontiguousarray(vol.transpose(1, 2, 0, 3))
#vi = pgl.GLVolumeItem(vol)
#self.glView.addItem(vi)
#vi.translate(-vol.shape[0]/2., -vol.shape[1]/2., -vol.shape[2]/2.)



verts, faces = pg.isosurface(ndi.gaussian_filter(padded_img.astype('float32'), (2, 2, 2)), 5.0)
md = pgl.MeshData(vertexes=verts, faces=faces)
mesh = pgl.GLMeshItem(meshdata=md, smooth=False, color=[0.5, 0.5, 0.5, 0.2], shader='balloon')
mesh.setGLOptions('additive')
mesh.translate(-padded_img.shape[0]/2., -padded_img.shape[1]/2., -padded_img.shape[2]/2.)
view.addItem(mesh)

view.show()

In [9]:
ontology = mcc.get_ontology()

In [10]:
def get_connectivity(source_abb,targ_abb):
    '''
    Maps the connectivity between two structures of interest.
    Inputs:
        source_abb (str): abbreviation for the source structure.
        targ_abb (str): abbreviation for the target structure.
    Outputs:
        unionizes (dict): a dictionary of pandas tables for all the experiments where the source injections 
            resulted in expression within the target structure. Keys are the experiment IDs.
        npv (dict): a dictionary where the keys are the experiment IDs and the values are the 
            normalized projection volume of the source injection within the target structure.
    '''
    
    # get structure IDs
    source_ID = ontology.df[ontology.df.acronym==source_abb].id.values[0]
    targ_ID = ontology.df[ontology.df.acronym==targ_abb].id.values[0]
    
    # pull source experiments
    source = mcc.get_experiments(dataframe=True, injection_structure_ids=[source_ID])

    # identify all experiments with injections from source
    source_exp_IDs = list(source.id.values)
    
    unionizes = mcc.get_structure_unionizes(source.id.values, structure_ids=[source_ID,targ_ID])

    filtered_projection_data = unionizes[(unionizes.hemisphere_id == 3) &
                                         (unionizes.is_injection==False) &
                                        (unionizes.structure_id == targ_ID)]
    filtered_injection_data = unionizes[(unionizes.hemisphere_id == 3) &
                                         (unionizes.is_injection==True) &
                                        (unionizes.structure_id == source_ID)]
     
    return filtered_injection_data, filtered_projection_data

In [11]:
fid_PRNc_III, fpd_PRNc_III=get_connectivity('PRNc','III')

In [12]:
fpd_PRNc_III.experiment_id[0]

300691325

In [13]:
pdensity,_ = mcc.get_projection_density(fpd_PRNc_III.experiment_id[0])

In [28]:
# remove item
view.removeItem(vi)

In [ ]:
#b = gl.GLBoxItem()
#w.addItem(b)
g = pgl.GLGridItem()
g.scale(10, 10, 1)
view.addItem(g)

In [ ]:
view.show()

In [29]:
# render volume
vol = np.zeros(pdensity.shape + (4,), dtype='ubyte')
vol[...,3] = pdensity*255
vol[...,1] = 255
#vol = np.ascontiguousarray(vol.transpose(1, 2, 0, 3))
vi = pgl.GLVolumeItem(vol)
vi.translate(-vol.shape[0]/8., -vol.shape[1]/8., -vol.shape[2]/8.)
vi.scale(1./4,1./4,1./4)
view.addItem(vi)


# verts_pd, faces_pd = pg.isosurface(ndi.gaussian_filter(pd.astype('float32'), (2, 2, 2)), .05)
# md_d = pgl.MeshData(vertexes=verts_pd, faces=faces_pd)
# mesh_d = pgl.GLMeshItem(meshdata=md_d, smooth=True, color=[1, 0.5, 1, 0.9], shader='balloon')
# mesh_d.setGLOptions('additive')
# mesh_d.translate(-pd.shape[0]/2., -pd.shape[1]/2., -pd.shape[2]/2.)
# view.addItem(mesh_d)

view.show()

In [30]:
vi.rotate(-90, 1, 0, 0)
mesh.rotate(-90, 1, 0, 0)

In [33]:
import time

In [38]:


for i in range(360):
    view.orbit(1,0)
    time.sleep(0.01)
    pg.QtGui.QApplication.processEvents()